# Process Data
> By Gati Aher  
> Sept 17, 2021

**Dataset:** FCF Carbon Perturbation (Cellulose-Glucose-Malate)

**Goal:**
- Process raw data for downstream analysis
- Put data in more workable format

In [1]:
# imports
import skbio
import pandas as pd
import numpy as np
import re

## Load Raw Data

Inputs:
* Sample ID: Sample Name
    * (Jean): `sample_names.csv`
* Sample Name x Meta Information
    * (Jean): `FCF_annotations.csv`
* OTU ID-Taxonomy x Sample Name x OTU Counts
    * (MR DNA): `081616JHnew515Fcomplete-pr.fasta.otus.fa.OTU.txt`
* OTU ID x Functions
    * (Shree)

In [2]:
# get map of MR DNA sample IDs to sample names
sample_names = pd.read_csv("../data/raw/sample_names.csv", index_col=0)
# prefix sample names with s (so they do not start with a number) 
sample_names['key'] = 's' + sample_names['key'].astype(str)

sample_names

,key
name,
Olin1,sC0C
Olin2,s1C03A
Olin3,s1C03B
Olin4,s1G03A
Olin5,s1G03B
...,...
Olin84,s3G10A
Olin85,s3G10B
Olin86,s3G10C


In [3]:
# get table of sample names to sample meta data
sample_annotations_from_file = pd.read_csv("../data/raw/FCF_annotations.csv", index_col=0) # these aren't in the right order
sample_annotations_from_file

,series,food,day,state
C0C,C0,cellulose,10,stable
1C03A,1C,cellulose,3,early
1C03B,1C,cellulose,3,early
1C05A,1C,cellulose,5,transitioning
1C05B,1C,cellulose,5,early
...,...,...,...,...
3G07A,3G,cellulose,7,stabilizing
3G07B,3G,cellulose,7,stabilizing
3G07C,3G,cellulose,7,stabilizing
3G10A,3G,cellulose,10,stable


In [4]:
# get table of OTU ID-taxonomy x absolute OTU counts
otu_raw = pd.read_csv("../data/raw/081616JHnew515Fcomplete-pr.fasta.otus.fa.OTU.txt", sep="\t", index_col=0)
otu_raw.index.names = ['featureID']
otu_raw.head()

,16rRNA,Percent Homology,evalue,bitscore,homology extract gibbsfree_otu/gibbsfree_homolog,total length gibbsfree_otu/otu_lenth/gibbsfree_homolog/homolog_length,Olin1,Olin2,Olin3,Olin4,...,Olin80,Olin81,Olin82,Olin83,Olin84,Olin85,Olin86,Olin87,Olin88,Unnamed: 95
featureID,,,,,,,,,,,,,,,,,,,,,
OTU_1,eu528231.1 seasonal dynamics mudflat mouth maj...,100.000000,0.0,711.813612,NaN,NaN,6,11,13,20,...,11,19,25,22,16,16,6,5,4,NaN
OTU_2,uncultured opitutus sp. ;k__bacteria;p__verruc...,96.969697,0.0,661.319285,NaN,NaN,17546,18302,32674,9700,...,31216,17838,12257,19679,16779,15614,2107,42046,56298,NaN
OTU_3,rhodopseudomonas palustris ;k__bacteria;p__pro...,99.493671,0.0,704.600137,NaN,NaN,3169,3033,4495,3455,...,1073,4207,1275,2274,4900,2027,1140,3192,2086,NaN
OTU_4,ay297802.1 waterlogged archaeological wood clo...,99.492386,0.0,702.796768,NaN,NaN,5243,3526,4658,29020,...,17928,19104,11308,11368,26958,15939,3121,30080,23869,NaN
OTU_5,rhodoblastus acidophilus ;k__bacteria;p__prote...,99.746835,0.0,708.206874,NaN,NaN,3809,1128,1313,1001,...,17129,22577,7156,26149,9243,2211,1368,13865,5190,NaN


In [5]:
# get table of OTU functions
# TODO

## Create and Save Processed Data

Outputs:
* Sample Name x Meta Information
    * `sample_metadata.tsv`
* OTU ID x Sample ID x HTS Counts
    * `OTU_counts.tsv`
* OTU ID x Taxonomy, Functions
    * `feature_metadata.tsv`
* OTU ID x Taxonomy
    * `taxonomy.tsv`

### Create `sample_metadata.tsv`

In [7]:
# format useful sample annotations file
# columns: index series carbon transfer day

ann_index = []
ann_series = []
ann_carbon = []
ann_transfer = []
ann_group = []
ann_day = []
ann_replicate = []
ann_state = []

for sn in sample_annotations_from_file.index:
    # index
    ann_index.append("s" + sn)
    # series
    if sn == "C0C":
        ann_series.append("C0C")
    else:
        ann_series.append("s" + sn[1])
    # carbon
    if sn == "C0C":
        ann_carbon.append("original")
    elif sample_annotations_from_file.loc[sn, "food"] == "cellulose":
        ann_carbon.append("cellulose")
    elif sample_annotations_from_file.loc[sn, "food"] == "glucose":
        ann_carbon.append("glucose")
    elif sample_annotations_from_file.loc[sn, "food"] == "malate":
        ann_carbon.append("malate")
    else:
        ann_carbon.append("X")
    # transfer
    if sn == "C0C":
        ann_transfer.append(0)
    else:
        ann_transfer.append(int(sn[0]))
    # group
    if sn == "C0C":
        ann_group.append("C0C")
    else:
        ann_group.append(sn[0:2])
    # day
    if sn == "C0C":
        ann_day.append(10)
    else:
        ann_day.append(int(sn[2:-1]))
    # replicate
    if sn == "C0C":
        ann_replicate.append("C0C")
    else:
        ann_replicate.append(sn[-1])  
    # state (from unsupervised clustering)
    ann_state.append(sample_annotations_from_file.loc[sn, "state"])
        
# print("ann_group", ann_group)
# print("ann_series", ann_series)
# print("ann_carbon", ann_carbon)
# print("ann_transfer", ann_transfer)
# print("ann_day", ann_day)
# print("ann_replicate", ann_replicate)
# print("ann_index", ann_index)

sample_metadata = pd.DataFrame()
sample_metadata["group"] = ann_group
sample_metadata["series"] = ann_series
sample_metadata["carbon"] = ann_carbon
sample_metadata["transfer"] = ann_transfer
sample_metadata["day"] = ann_day
sample_metadata["replicate"] = ann_replicate
sample_metadata["state"] = ann_state
sample_metadata["sampleID"] = ann_index
sample_metadata = sample_metadata.set_index("sampleID")

print("samples:", list(sample_metadata.group))

samples: ['C0C', '1C', '1C', '1C', '1C', '1C', '1C', '1C', '1C', '1C', '1C', '2C', '2C', '2C', '2C', '2C', '2C', '1M', '1M', '1M', '1M', '1M', '1M', '1M', '1M', '1M', '1M', '1M', '1M', '2M', '2M', '2M', '2M', '2M', '2M', '2M', '2M', '2M', '2M', '2M', '3M', '3M', '3M', '3M', '3M', '3M', '3M', '3M', '3M', '3M', '3M', '3M', '1G', '1G', '1G', '1G', '1G', '1G', '1G', '1G', '1G', '1G', '1G', '1G', '2G', '2G', '2G', '2G', '2G', '2G', '2G', '2G', '2G', '2G', '2G', '2G', '3G', '3G', '3G', '3G', '3G', '3G', '3G', '3G', '3G', '3G', '3G']


In [8]:
sample_metadata = sample_metadata.drop(["s2M07C"])
sample_metadata

,group,series,carbon,transfer,day,replicate,state
sampleID,,,,,,,
sC0C,C0C,C0C,original,0,10,C0C,stable
s1C03A,1C,sC,cellulose,1,3,A,early
s1C03B,1C,sC,cellulose,1,3,B,early
s1C05A,1C,sC,cellulose,1,5,A,transitioning
s1C05B,1C,sC,cellulose,1,5,B,early
...,...,...,...,...,...,...,...
s3G07A,3G,sG,cellulose,3,7,A,stabilizing
s3G07B,3G,sG,cellulose,3,7,B,stabilizing
s3G07C,3G,sG,cellulose,3,7,C,stabilizing


In [9]:
type(sample_metadata["day"][1])

numpy.int64

In [10]:
# save
sample_metadata.to_csv("../data/processed/sample_metadata.tsv", sep='\t')

### Create `OTU_counts.tsv`

In [ ]:
# format table of OTU ID x sample name x absolute counts
OTU_counts = otu_raw.filter(regex='^Olin')
# rename sample ID to sample names
OTU_counts = OTU_counts.rename(columns=sample_names.to_dict()["key"])
# NOTE: 3G10C and 2M7C were removed because they were incorrectly sequenced
OTU_counts = OTU_counts.drop(columns=["s3G10C", "s2M07C"])
# order columns by order in sample metadata
OTU_counts = OTU_counts.reindex(columns=list(sample_metadata.index))

In [ ]:
OTU_counts

In [ ]:
OTU_counts.to_csv("../data/processed/OTU_counts.tsv", sep='\t')

### Create `feature_metadata.tsv` 

In [ ]:
# format table of OTU ID x taxonomy & function
feature_metadata = pd.DataFrame(index=otu_raw.index, columns=["Name"])

for otu, desc in otu_raw.iterrows(): 
    parts = desc["16rRNA"].split(";")
    taxonomy = "Unassigned"
    for i, part in enumerate(parts):
        if i == 0:
            feature_metadata.loc[otu, "Name"] = part
        elif part.startswith("k__"):
            feature_metadata.loc[otu, "kingdom"] = part[3:]
            taxonomy = "Root;" + part + ";"
        elif part.startswith("p__"):
            feature_metadata.loc[otu, "phylum"] = part[3:]
            taxonomy += part + ";"
        elif part.startswith("c__"):
            feature_metadata.loc[otu, "class"] = part[3:]
            taxonomy += part + ";"
        elif part.startswith("o__"):
            feature_metadata.loc[otu, "order"] = part[3:]
            taxonomy += part + ";"
        elif part.startswith("f__"):
            feature_metadata.loc[otu, "family"] = part[3:]
            taxonomy += part + ";"
        elif part.startswith("g__"):
            feature_metadata.loc[otu, "genus"] = part[3:]
            taxonomy += part + ";"
        elif part.startswith("s__"):
            feature_metadata.loc[otu, "species"] = part[3:]
            taxonomy += part + ";"
        elif part.startswith("superkingdom__"):
            feature_metadata.loc[otu, "superkingdom"] = part[14:]
        elif part.startswith("superphylum__"):
            feature_metadata.loc[otu, "superphylum"] = part[13:]
        elif part.startswith("subclass"):
            feature_metadata.loc[otu, "subclass"] = part[8:]
        elif part.startswith("suborder"):
            feature_metadata.loc[otu, "suborder"] = part[8:]
        elif part.startswith("subphylum__"):
            feature_metadata.loc[otu, "subphylum"] = part[9:]
        else:
            print("OTU:", otu, "Extra PART:", part)
    feature_metadata.loc[otu, "taxonomy"] = taxonomy

In [ ]:
feature_metadata

In [ ]:
# save
feature_metadata.to_csv("../data/processed/feature_metadata.tsv", sep='\t')

### Create `taxonomy.tsv`

In [ ]:
# save
feature_metadata["taxonomy"].to_csv("../data/processed/taxonomy.tsv", sep='\t')
feature_metadata["taxonomy"]

### Additional Cleaning

In [ ]:
# check if there are OTUs that appear in no samples
OTU_counts[OTU_counts.sum(axis=1) == 0]

In [ ]:
# remove OTUs that are not bacteria
not_bacteria = feature_metadata[feature_metadata["kingdom"] != "bacteria"]
print(not_bacteria.Name)

OTU_counts_clean = OTU_counts.drop(not_bacteria.index)
print("OTU_counts.shape", OTU_counts.shape, "OTU_counts_clean.shape", OTU_counts_clean.shape,)

feature_metadata_clean = feature_metadata.drop(not_bacteria.index)
print("feature_metadata.shape", feature_metadata.shape, "feature_metadata_clean.shape", feature_metadata_clean.shape,)

In [ ]:
not_bacteria.shape

In [ ]:
# remove OTUs with less than threshold total reads
threshold_OTU_total_reads = 5
less_than_threshold = OTU_counts_clean[OTU_counts_clean.sum(axis=1) < threshold_OTU_total_reads]
print(less_than_threshold.sum(axis=1))

OTU_counts_clean = OTU_counts_clean.drop(less_than_threshold.index)
print("OTU_counts_clean.shape", OTU_counts_clean.shape,)

feature_metadata_clean = feature_metadata_clean.drop(less_than_threshold.index)
print("feature_metadata_clean.shape", feature_metadata_clean.shape,)

In [ ]:
# save clean
OTU_counts_clean.to_csv("../data/processed/OTU_counts_clean.tsv", sep='\t')
feature_metadata_clean.to_csv("../data/processed/feature_metadata_clean.tsv", sep='\t')
feature_metadata_clean[["kingdom", "phylum", "class", "order", "family", "genus", "species"]].to_csv("../data/processed/taxonomy_table_clean.tsv", sep='\t')

## Get Phylogenetic Info and Alignment Scores

* OTU ID: DNA Sequences
    * (MR DNA): `081616JHnew515Fcomplete-pr.fasta.otus.fa`
* OTU ID x OTU ID Pairwise Alignment Scores
    * (Muscle): `Pairwise_distances_all_OTU_Muscle`
* OTU ID Phylogenetic Tree
    * (Muscle): `MUSCLE_alignment_ML_tree_all_OTUs`

### Rename OTU ID: DNA Sequences mapping (for Jean, to visually analyze phylogenetic tree)

In [ ]:
from skbio.sequence import DNA

sequence_file = open("../data/raw/081616JHnew515Fcomplete-pr.fasta.otus.fa", 'r')
sequence_lines = [l.upper() for l in sequence_file.readlines()]
dna_lines = [DNA(str(seq), metadata=seq.metadata) for seq in skbio.io.read(sequence_lines, format='fasta')]
dna_lines_renamed = []

for seq in skbio.io.read(sequence_lines, format='fasta'):
    str_seq = str(seq)
    metadata = seq.metadata
    metadata['id'] = metadata['id'] + "_" + feature_metadata.loc[metadata['id'], "Name"].replace(" ", "_").replace(".", "")
    dna_lines_renamed.append(DNA(str_seq, metadata=metadata))

In [ ]:
# save
dna_lines_renamed_file = open("../data/processed/renamed_081616JHnew515Fcomplete-pr.fasta.otus.fa", "w")
for seq in dna_lines_renamed:
    skbio.io.write(seq, into=dna_lines_renamed_file, format='fasta')

### Import and Use Newick Phylogenetic Tree

In [ ]:
# import phylogenetic weights and use in an example
from skbio import TreeNode
tree_file = open("../data/raw/MUSCLE_alignment_ML_tree_all_OTUs", "r")
tree = TreeNode.read(tree_file)
tree_rooted = tree.root_at_midpoint()
print(str(tree)[:50], "...")

In [ ]:
# alpha diversity w/ phylogenetic weights example 
from skbio.diversity import alpha_diversity
alpha_diversity(metric="faith_pd", counts=OTU_counts.T.values, ids=OTU_counts.columns, tree=tree_rooted, otu_ids=[n.name for n in tree_rooted.tips()], validate=True)

# Experiment Scratch-Pad